<a href="https://colab.research.google.com/github/danielbauer1860/ddg_project/blob/main/Niederschlagsmenge_Deutschland_LinReg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introductory information

Drei Regensburger Stationen: 

- 03578 : Regensburg-Burgweinting
- 04104 : Regensburg
- 04105 : Regensburg-Obertraubling (keine aktuellen Daten)

#Data Pre-Processing

In [ ]:
import pandas as pd
import numpy as np
import math as mh
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [ ]:
df = pd.read_csv("04104-niederschlag.txt", header=0, sep=";")
df.drop(columns=['STATIONS_ID', 'MO_NSH', 'MO_SH_S', 'eor', 'MESS_DATUM_ENDE'], inplace=True)

df['MESS_DATUM_BEGINN'] = pd.to_datetime(df['MESS_DATUM_BEGINN'], format="%Y%m%d")
df.set_index('MESS_DATUM_BEGINN', inplace=True)
df.head()

,QN_6,MO_RR,MX_RS
MESS_DATUM_BEGINN,,,
2021-05-01,9,96.0,30.7
2021-06-01,9,77.1,12.8
2021-07-01,9,105.9,23.5
2021-08-01,9,101.7,34.4
2021-09-01,3,15.5,3.8


In [ ]:
df['PCT_CHANGE'] = df['MO_RR'].pct_change()
df.tail()

,QN_6,MO_RR,MX_RS,PCT_CHANGE
MESS_DATUM_BEGINN,,,,
2022-07-01,3,11.4,2.9,-0.877419
2022-08-01,3,70.5,19.1,5.184211
2022-09-01,1,91.3,23.4,0.295035
2022-10-01,1,44.0,10.5,-0.518072
2022-11-01,1,-999.0,23.4,-23.704545


In [ ]:
df['Label'] = np.nan
df.tail()

,QN_6,MO_RR,MX_RS,PCT_CHANGE,Label
MESS_DATUM_BEGINN,,,,,
2022-07-01,3,11.4,2.9,-0.877419,NaN
2022-08-01,3,70.5,19.1,5.184211,NaN
2022-09-01,1,91.3,23.4,0.295035,NaN
2022-10-01,1,44.0,10.5,-0.518072,NaN
2022-11-01,1,-999.0,23.4,-23.704545,NaN


In [ ]:
forecastRange = int(mh.ceil(0.01 * len(df)))
df.fillna(value=0, inplace=True)
df['Label'] = df['MO_RR'].shift(-forecastRange)
df.tail()

,QN_6,MO_RR,MX_RS,PCT_CHANGE,Label
MESS_DATUM_BEGINN,,,,,
2022-07-01,3,11.4,2.9,-0.877419,70.5
2022-08-01,3,70.5,19.1,5.184211,91.3
2022-09-01,1,91.3,23.4,0.295035,44.0
2022-10-01,1,44.0,10.5,-0.518072,-999.0
2022-11-01,1,-999.0,23.4,-23.704545,NaN


In [ ]:
df.replace(-999.0, np.nan, inplace=True)
df.dropna(inplace=True)
df.tail()

,QN_6,MO_RR,MX_RS,PCT_CHANGE,Label
MESS_DATUM_BEGINN,,,,,
2022-05-01,3,62.7,14.0,0.271805,93.0
2022-06-01,3,93.0,22.2,0.483254,11.4
2022-07-01,3,11.4,2.9,-0.877419,70.5
2022-08-01,3,70.5,19.1,5.184211,91.3
2022-09-01,1,91.3,23.4,0.295035,44.0


#Creating the model



In [ ]:
X = np.array(df.drop(['Label'], axis=1))
y = np.array(df['Label'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

In [ ]:
model = LinearRegression()

In [ ]:
model.fit(X_train, y_train)
model.score(X_test, y_test)

-0.410238542853957

#Prediction and visualisation

In [ ]:
X_PredictionSet = X[-forecastRange:]
X_PredictionSet

array([[ 1.        , 91.3       , 23.4       ,  0.29503546]])

In [ ]:
forecastSet = model.predict(X_PredictionSet)
forecastSet

array([20.77311697])

In [ ]:
import datetime
from matplotlib import style

Quellen:
https://opendata.dwd.de/climate_environment/